In [28]:
fs = require('fs');
dsv = require('d3-dsv');
simpleStats = require('simple-statistics');
_ = require('lodash');
csv = require('csv-parser');

''

''

In [29]:
ct_census = dsv.csvParse(fs.readFileSync('too_big/ss14hct.csv', 'utf8'));
// from http://www2.census.gov/programs-surveys/acs/data/pums/2014/1-Year/csv_hct.zip

''

''

In [30]:
// these PUMA IDs represent the Bridgeport-Stamford-Norwalk, CT (Metro) according to geocorr
bridgeport_2014 = ct_census.filter(d => +d.PUMA >= 100 && +d.PUMA <= 105);

/*
"ADJINC
Adjustment factor for income and earnings dollar amounts(6 implied decimal places)
1008425 .2014 factor (1.008425)

Note: The value of ADJINC inflation-adjusts reported income to 2014 dollars.
ADJINC applies to variables FINCP and HINCP in the housing record, and variables
INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, and WAGP in the person record. 

HINCP
Household income (past 12 months)
bbbbbbbbb .N/A(GQ/vacant)
000000000 .No household income
-00059999 .Loss of -$59,999 or more
-00000001..-00059998 .Loss of $1 to -$59,998
000000001 .$1 or Break even
000000002..999999999 .Total household income in dollars (Components are rounded)

Note: Use ADJINC to adjust HINCP to constant dollars."

https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict14.txt
*/

bridgeport_2014 = bridgeport_2014.filter(d => d.HINCP !== ' ')
bridgeport_2014.forEach(d => d.ADJHH = (d.ADJINC/1000000)*d.HINCP);

''

''

In [31]:
function multiply (row, income_col, weight_col, puma_col) {
    let result = [];
    for (let i = 0; i < +row[weight_col]; i++) {
        // let resultRow = {};
        // resultRow[income_col] = row[income_col];
        // resultRow[puma_col] = row[puma_col];
        result.push(row[income_col]);
    }
    return result;
}

/*
Make a list of HHINCOME for calculating
a quantile where each household's HHINCOME
is repeated in the series n times, where
n == weight_col
*/
function explode_weights (rows, income_col, weight_col, puma_col) {
    let result = [];
    for (let row of rows) {
        result = result.concat(multiply(row, income_col, weight_col, puma_col));
    }
    return result;
}

exploded_bridgeport = explode_weights(bridgeport_2014, 'ADJHH', 'WGTP', 'PUMA');

''

''

In [32]:
simpleStats.quantile(exploded_bridgeport,0.95); // .map(d => d.ADJHH)

558969.9774999999

# Hallelujah!

In [33]:
acs_16 = dsv.csvParse(fs.readFileSync('hh_16.csv', 'latin1'));
mable = dsv.csvParse(fs.readFileSync('geocorr14_2014.csv', 'latin1')).slice(1);

top_coded = acs_16.filter(d => !d.HD01_VD06)
bottom_coded = acs_16.filter(d => !d.HD01_VD02)

top_coded.forEach(d => d.States = d['GEO.display-label'].split(', ')[1].split(' ')[0].split('-'))

In [34]:
function make_stpuma(st, puma) {
    return +(st + ('00000'+puma).slice(-5));
}
make_stpuma(9,102)

900102

In [35]:
ct_census.forEach(d => d.stpuma = make_stpuma(d.ST,d.PUMA));

In [36]:
mable.forEach(d => d.stpuma = make_stpuma(d.state, d.puma12))
mable.slice(0,5)

[ { state: '01',
    puma12: '00100',
    cbsa: ' ',
    stab: 'AL',
    cbsaname15: ' ',
    PUMAname: 'Lauderdale, Colbert, Franklin & Marion (Northeast) Counties',
    pop14: '39326.125',
    afact: '0.21 ',
    stpuma: 100100 },
  { state: '01',
    puma12: '00100',
    cbsa: '22520',
    stab: 'AL',
    cbsaname15: 'Florence-Muscle Shoals, AL (Metro)',
    PUMAname: 'Lauderdale, Colbert, Franklin & Marion (Northeast) Counties',
    pop14: '147639',
    afact: '0.79 ',
    stpuma: 100100 },
  { state: '01',
    puma12: '00200',
    cbsa: '26620',
    stab: 'AL',
    cbsaname15: 'Huntsville, AL (Metro)',
    PUMAname: 'Limestone & Madison (Outer) Counties--Huntsville City (Far West & Southwest)',
    pop14: '183944.849',
    afact: '1 ',
    stpuma: 100200 },
  { state: '01',
    puma12: '00301',
    cbsa: '26620',
    stab: 'AL',
    cbsaname15: 'Huntsville, AL (Metro)',
    PUMAname: 'Huntsville (North) & Madison (East) Cities',
    pop14: '124425.297',
    afact: '1 ',
    stpuma

In [37]:
function get_pumas(geo_id, geo_corr) {
    cbsa = geo_corr.filter(d => +d.cbsa == +geo_id);
    target = {};
    for (let row of cbsa) {
        target[row.stpuma] = parseFloat(row.afact); // allocation factor (proportion of PUMA within in the geography)
    }
    return target;
}

top_coded.forEach(d => d.PUMAS = get_pumas(d['GEO.id2'], mable));
all_pumas = _(top_coded).map('PUMAS').flatMap(_.keys).uniq().value();

''

''

In [38]:
// this appears not to be used
// state_fips = _(mable).groupBy('stab').mapValues(d => _.maxBy(d,'state')).value();
// state_fips.CA.state

In [39]:
/*
axios = require('axios');
// import zipfile,requests,io,os

function fetch_census(x, direct):
    let zip_path = `https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_h${x}.zip`;
    r = requests.get(zip_path, stream=True)
    
    axios.get(zip_path, { responseType: 'stream' })
        .then(response => {
            response.data.pipe(fs.createWriteStream('ada_lovelace.jpg'))
        });

    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    insert = ""
    if direct is not None:
        insert = direct+'/'
    dir_path = "too_big/"+insert+"2016/csv_h"+x+"/"
    zip_ref.extractall(dir_path)
    os.remove("too_big/"+insert+"2016/csv_h"+x+"/ACS2016_PUMS_README.pdf")

def download_unzip_census(ss_s,direct=None):
    ss_s.apply(lambda x: fetch_census(x.lower(),direct))
*/

function make_path(year, state, direct) {
  return `too_big/${direct
    ? direct + '/'
    : ''}${year}/csv_h${state.toLowerCase()}/ss16h${state.toLowerCase()}.csv`;
}

function get_census_pums_file(s,is_rel=[],direct=null) {
    console.log(s);
    let result = [];
    return new Promise (function (resolve, reject) {
        let file = make_path(2016,s,direct)
        
        fs.createReadStream(file)
            .pipe(csv())
            .on('data', d => {
                let stpuma = make_stpuma(d.ST,d.PUMA);
                if (is_rel.indexOf("" + stpuma) !== -1) {
                    result.push({
                        stpuma,
                        HINCP: d.HINCP,
                        WGTP: d.WGTP
                    });
                }
            })
            .on('end', d => resolve(result));
    });
}


async function get_census_pums(ss_s,is_rel=[],direct=null) {
    let result = [];
    for (s of ss_s) {
        result = result.concat(await get_census_pums_file(s,is_rel,direct=null));
    }
    return result;
}

In [40]:
pop_16 = dsv.csvParse(fs.readFileSync('ACS_16_1YR_B01003.csv', 'latin1')); // total population by metro area

top_coded = top_coded.filter(d =>
    (d['GEO.display-label'].indexOf('Metro') !== -1) && // filter to metro areas only
    (d['GEO.display-label'].indexOf(' PR ') === -1) && // this might do nothing?
    (+pop_16.find(d2 => d['GEO.id2'] == d2['GEO.id2']).HD01_VD01 >= 500000) // population at or over 500,000
)

''

''

In [41]:
async function pums_dataset (tc, is_rel, states_dir=null) {
    s = _(tc).flatMap('States').uniq().sort().value();
    console.log(s);
    // if not (os.path.isfile(make_path(2016,sls[0],states_dir))):
    //    download_unzip_census(s,states_dir)
    relevant_census = await get_census_pums(s,is_rel,states_dir);
    relevant_census = relevant_census.filter(d => d.HINCP !== '');
    // relevant_census.forEach(d => d.ADJHH = (d.ADJINC/1000000)*d.HINCP);
    // exploded_census = explode_weights(relevant_census, 'HHINC', 'WGTP', 'stpuma');
    return relevant_census;
}

$$.async();

relevant_census = [];

(function($$) {
    pums_dataset(top_coded, all_pumas)
        .then(result => {
            relevant_census = result;

            $$.done();
        });
})($$);


[ 'CA', 'CT', 'DC', 'MA', 'MD', 'NH', 'NJ', 'NY', 'PA', 'VA', 'WV' ]
CA
CT
DC
MA
MD
NH
NJ
NY
PA
VA
WV


In [42]:
x = []
for (value of top_coded) {
    for (entry of Object.keys(value.PUMAS)) {
        x.push({
            puma: entry,
            afact: +value.PUMAS[entry] // allocation factor (proportion of PUMA within in the geography)
        });
    }
}

df = x.slice(1)

top_coded.length

8

In [43]:
_(df).groupBy(d => d.afact<1).mapValues(d => d.length).value();

{ false: 369, true: 15 }

In [44]:
// is this the right way to do population interpolation?
function get_data(puma, percent, census) {
    let all_entries = explode_weights(census.filter(d => +d.stpuma == +puma), 'HINCP', 'WGTP', 'stpuma');
    return percent < 1
        ? simpleStats.sample(all_entries, all_entries.length * percent) // seed?
        : all_entries;
}
    
function make_fips_data_dict(df,census) {
    let d = {};
    for (let row of df) {
        let result = [];
        for (p of Object.keys(row.PUMAS)) {
            data = get_data(p,row.PUMAS[p],census);
            result = result.concat(data);
        }
        d[row['GEO.id2']] = result;
    }
    return d;
}


fips_data_dict = make_fips_data_dict(top_coded,relevant_census);

''

''

In [45]:
function get_quantile(fips,percent,data) {
    return simpleStats.quantile(data[fips].map(d => +d),percent);
}

function adjinc(ss,year) {
    let file = make_path(year,ss,null);
    let result = dsv.csvParse(fs.readFileSync(file, 'utf8'));
    return result[0].ADJINC;
}

adj_inc = adjinc('ct',2016)

top_coded.forEach(d => d['95_Percentile'] = get_quantile(d['GEO.id2'],.95,fips_data_dict)*adj_inc/1000000)
top_coded.forEach(d => d['20_Percentile'] = get_quantile(d['GEO.id2'],.2,fips_data_dict)*adj_inc/1000000)

''

''

In [46]:
acs_16.slice(0,3);

[ { 'GEO.id': '1600000US0640354',
    'GEO.id2': '0640354',
    'GEO.display-label': 'La Quinta city, California',
    HD01_VD02: '27794',
    HD02_VD02: '2167',
    HD01_VD03: '55920',
    HD02_VD03: '4923',
    HD01_VD04: '90161',
    HD02_VD04: '6006',
    HD01_VD05: '146206',
    HD02_VD05: '6942',
    HD01_VD06: '',
    HD02_VD06: '',
    States: [ 'California' ],
    PUMAS: {} },
  { 'GEO.id': '310M300US10100',
    'GEO.id2': '10100',
    'GEO.display-label': 'Aberdeen, SD Micro Area',
    HD01_VD02: '24106',
    HD02_VD02: '2012',
    HD01_VD03: '43064',
    HD02_VD03: '2586',
    HD01_VD04: '68480',
    HD02_VD04: '3310',
    HD01_VD05: '100118',
    HD02_VD05: '4226',
    HD01_VD06: '178615',
    HD02_VD06: '19617' },
  { 'GEO.id': '310M300US10140',
    'GEO.id2': '10140',
    'GEO.display-label': 'Aberdeen, WA Micro Area',
    HD01_VD02: '19637',
    HD02_VD02: '1746',
    HD01_VD03: '35832',
    HD02_VD03: '1603',
    HD01_VD04: '55115',
    HD02_VD04: '2873',
    HD01_VD05:

In [64]:
function ninetyfive_twenty(row) {
    if (+row['GEO.id2'] == 640354) {
        return NaN;
    }
    if (row['HD01_VD02'] == '') {
        return NaN;
    }
    if (row['HD01_VD06'] == '' || isNaN(+row['HD01_VD06'])) {
        return (+row['95_Percentile']/+row['20_Percentile']);
    }
    else {
        return (+row['HD01_VD06']/+row['HD01_VD02']);
    }
}

inequality_chart = acs_16.map(d => {
    let top_row = top_coded.find(d2 => +d2['GEO.id2'] == +d['GEO.id2']);
    
    return {
        'GEO.id2': d['GEO.id2'],
        'GEO.display-label': d['GEO.display-label'],
        'HD01_VD02': d['HD01_VD02'],
        'HD01_VD06': d['HD01_VD06'],
        '95_Percentile': top_row ? top_row['95_Percentile'] : NaN,
        '20_Percentile': top_row ? top_row['20_Percentile'] : NaN
    };
});

inequality_chart = inequality_chart.filter(d => d);

inequality_chart.forEach(d => d['9520'] = ninetyfive_twenty(d));

inequality_chart = inequality_chart.filter(d => d && d['9520']);

''

''

In [65]:
inequality_chart = _.orderBy(inequality_chart,'9520','desc');

[ { '9520': 21.455577632699406,
    'GEO.id2': '32420',
    'GEO.display-label': 'Mayagüez, PR Metro Area',
    HD01_VD02: '3523',
    HD01_VD06: '75588',
    '95_Percentile': NaN,
    '20_Percentile': NaN },
  { '9520': 19.04336966394187,
    'GEO.id2': '42180',
    'GEO.display-label': 'Santa Isabel, PR Micro Area',
    HD01_VD02: '4404',
    HD01_VD06: '83867',
    '95_Percentile': NaN,
    '20_Percentile': NaN },
  { '9520': 15.5657908847185,
    'GEO.id2': '24900',
    'GEO.display-label': 'Greenwood, MS Micro Area',
    HD01_VD02: '9325',
    HD01_VD06: '145151',
    '95_Percentile': NaN,
    '20_Percentile': NaN },
  { '9520': 15.109747023809524,
    'GEO.id2': '44260',
    'GEO.display-label': 'Starkville, MS Micro Area',
    HD01_VD02: '10752',
    HD01_VD06: '162460',
    '95_Percentile': NaN,
    '20_Percentile': NaN },
  { '9520': 14.742054981477871,
    'GEO.id2': '35060',
    'GEO.display-label': 'Natchitoches, LA Micro Area',
    HD01_VD02: '10258',
    HD01_VD06: '15122

In [66]:
inequality_chart.filter(d => {
    let pop_row = pop_16.find(d2 => +d2['GEO.id2'] == +d['GEO.id2']);
    
    return pop_row && +pop_row['HD01_VD01']>=500000;
});

[ { '9520': 14.176470588235295,
    'GEO.id2': '14860',
    'GEO.display-label': 'Bridgeport-Stamford-Norwalk, CT Metro Area',
    HD01_VD02: '32385',
    HD01_VD06: '',
    '95_Percentile': 485657.416,
    '20_Percentile': 34257.992 },
  { '9520': 11.984126984126984,
    'GEO.id2': '35620',
    'GEO.display-label': 'New York-Newark-Jersey City, NY-NJ-PA Metro Area',
    HD01_VD02: '24732',
    HD01_VD06: '',
    '95_Percentile': 304291.576,
    '20_Percentile': 25391.2176 },
  { '9520': 10.961111111111112,
    'GEO.id2': '41860',
    'GEO.display-label': 'San Francisco-Oakland-Hayward, CA Metro Area',
    HD01_VD02: '32235',
    HD01_VD06: '',
    '95_Percentile': 397594.2248,
    '20_Percentile': 36273.168 },
  { '9520': 10.76,
    'GEO.id2': '31080',
    'GEO.display-label': 'Los Angeles-Long Beach-Anaheim, CA Metro Area',
    HD01_VD02: '24426',
    HD01_VD06: '',
    '95_Percentile': 271041.172,
    '20_Percentile': 25189.7 },
  { '9520': 10.587463953425106,
    'GEO.id2': '35380'